### 2.4.1 - Tabulate the Popular Venues from each District

In [ ]:
#Extracting Venues & their respective coordinates (from helsinki_venues) using transposed-ranked venues array (rl_t) for each District into single Dataframe

ranked_venues_list = []
# Iterate Venue Categories (from transposed-ranked Venues array) by Rank
for rank in rl_t:
    # Iterate Ranked Categories w.r.t Districts in top10_venues
    for vc, d in zip(rank, top10_venues.District):
        # Filter empty dataframes
        if not (helsinki_venues.loc[(helsinki_venues['District'] == d) & (helsinki_venues['Venue Category'] == vc)]).empty:
            # Append extracted Dataframes (from expression) to ranked_venues_list
            ranked_venues_list.append(helsinki_venues.loc[(helsinki_venues['District'] == d) & (helsinki_venues['Venue Category'] == vc)]) 
        else:
            pass

ranked_venues_df = pd.concat(ranked_venues_list) # Merge (by concatenation) all extracted Dataframes into a single Dataframe
ranked_venues_df.reset_index(inplace=True, drop=True)

### 2.4.2 - Visualize Popular Venues in Helsinki City Map with Rank and Venue Category

In [ ]:
#Copy of 'Venue Category' values as 'Venue Category_RANK' to assign Rank Number
ranked_venues_df['Venue Category_RANK'] = ranked_venues_df['Venue Category'] 
ranked_venues_df.head()

In [ ]:
#Verify Districts list between top10_venues and ranked_venues_df
if (top10_venues.District.unique() == ranked_venues_df.District.unique()).any() is False:
    print('Ooops!')
else:
    print('All good!')
#list(ranked_venues_df[ranked_venues_df.District == dist].iloc[:, 6])

In [ ]:
# Extract values specific to 'Venue Category' with 'District' as filter from both dataframes (from ranked_venues_df & top10_venues)

ranks = range(1, 11) # Intialize Rank Numbers
venue_dists = top10_venues.District.to_list() # List Districts with Venues

for dist in venue_dists:
    vc_r = top10_venues[top10_venues.District == dist].values[0][1:] # Venue Categories from 'Venue Category_RANK' (top10_venues)
    vc = list(ranked_venues_df[ranked_venues_df.District == dist].iloc[:, 6]) # Venue Categories from 'Venue Category' (ranked_venues_df)
    
    
    # Assign Rank Number based on 'Venue Category_RANK' (from top10_venues)
    for r, rank in zip(vc_r, ranks): # Iterate Venue Categories and their respective Rank Numbers
        for v in vc:
            if r == v: # Equate 'Venue Category' with 'Venue Category_RANK' values and assign Rank Number
                ranked_venues_df['Venue Category_RANK'].replace(to_replace=v, value=rank, inplace=True) # Replace values in 'Venue Category_RANK' by Rank Number           

ranked_venues_df.head()

#### Investigating assigned Rank Numbers

In [ ]:
print(f"Total Null values in Venue Category_RANK: {ranked_venues_df['Venue Category_RANK'].isnull().sum()}")
print(f"List of Assigned Rank Numbers:\n{sorted(ranked_venues_df['Venue Category_RANK'].unique())}")

In [ ]:
top10_venues[top10_venues['District']=='Ullanlinna']

In [ ]:
ranked_venues_df[ranked_venues_df['District']=='Ullanlinna']

In [ ]:
# Districts with Popular Venues (Venues from Top10 Venue Categories for each District)
from folium import plugins

helsinki_popven_map = folium.Map(location=[latitude, longitude], zoom_start=12) # Initiate Helsinki City Map
pop_venues = plugins.MarkerCluster().add_to(helsinki_popven_map) # Initiate Cluster segmentation plugin

for dist, venue_cat, rank, lat, long in zip(ranked_venues_df['District'], ranked_venues_df['Venue Category'], ranked_venues_df['Venue Category_RANK'],
ranked_venues_df['Venue Latitude'], ranked_venues_df['Venue Longitude']):
    label = folium.Popup(f"{dist}, {venue_cat}, Rank-{rank}", parse_html=True)
    folium.Marker(location=[lat, long],
    icon=None,
    popup=label).add_to(pop_venues)

helsinki_popven_map

## TO DO: (24 Dec 2020)
* Create Dataframe for the Districts with Venues having:
    * Venue, Venue Latitude, Venue Longitude & Venue Category (from helsinki_venues)
    * Venue Categories (from top10_venues)
* Plot Venues in each District
* Calculate Cluster centre (centroid = Popularity Centre) for each District from Venue Latitude & Longitude
* Plot Popularity Centres on Helsinki City Map